# ImbalancedDomainsMetrics (Regression): Usage
## Example 2: Intermediate

## Dependencies
First, we load the required dependencies. Here we import regression_metrics from imbalanced_metrics to evalute the result we get from the XGBRegressor. In addition, we use pandas and numpy for data handling, and train_test_split to split the dataset.

In [13]:
## load dependencies
from imbalanced_metrics import regression_metrics as rm
import pandas as pd 
from xgboost import XGBRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error , r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


## Data
Firstly, we load our data. In this example, we use the Car Price Prediction Dataset retreived from Kaggle, originally complied by Manish Kumar. 
Link to original dataset - https://www.kaggle.com/datasets/hellbuoy/car-price-prediction

In [14]:
## load data
df = pd.read_csv(
    'https://raw.githubusercontent.com/paobranco/ImbalanceMetrics/main/data/CarPrice_Assignment.csv', index_col=None, na_values=['NA']
)

In [15]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


## Data preprocessing

In [16]:
# We need change the data type from object to number
object_data = df.select_dtypes(include='object')
num_data = df.select_dtypes(exclude='object')

enc = LabelEncoder()
for i in range(0, object_data.shape[1]):
    object_data.iloc[:,i] = enc.fit_transform(object_data.iloc[:,i])   

C:\Users\srtul\AppData\Local\Temp\ipykernel_20676\250724919.py:7: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  object_data.iloc[:,i] = enc.fit_transform(object_data.iloc[:,i])


In [17]:
# Merging the numerical part of the dataset and processed obeject part of the dataset together.
df = pd.concat([num_data, object_data], axis = 1)

In [18]:
# Show the description of the dataset
df.describe()

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,...,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,enginetype,cylindernumber,fuelsystem
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,...,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,103.000000,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,...,77.209756,0.902439,0.180488,0.439024,2.614634,1.326829,0.014634,3.014634,2.117073,3.253659
std,59.322565,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,...,41.014583,0.297446,0.385535,0.497483,0.859081,0.556171,0.120377,1.054765,0.795792,2.013204
min,1.000000,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,52.000000,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,...,44.000000,1.000000,0.000000,0.000000,2.000000,1.000000,0.000000,3.000000,2.000000,1.000000
50%,103.000000,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,...,80.000000,1.000000,0.000000,0.000000,3.000000,1.000000,0.000000,3.000000,2.000000,5.000000
75%,154.000000,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,...,115.000000,1.000000,0.000000,1.000000,3.000000,2.000000,0.000000,3.000000,2.000000,5.000000
max,205.000000,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,...,146.000000,1.000000,1.000000,1.000000,4.000000,2.000000,1.000000,6.000000,6.000000,7.000000


In [19]:
# Assign x and y values from the dataframe as train and test.
X = df.drop(columns="price") # Every other columns except price      
y = df["price"]    # y = price


In [20]:
#Split X and y into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

## Model
After, we train our model with data. In this example, we use the `XGBRegressor()` from sklearn


In [21]:
reg = XGBRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_test)

## phi relevance arguments
    

In this example, we dive a bit deeper. In beginer example, we left all the default values for `phi` relevance function. In this example, we are going pass few arguments to control the `phi` relevance. The arguments are given below :

* The `rel_method` argument takes a string, either `'auto'` or `'manual'`. It specifies how relevant or rare "minority" values in `y` are determined. If `'auto'` is specified, "minority" values are automatically determined by box plot extremes. If `'manual'` is specified, "minority" values are determined by the user. In this example, we input `'auto'` with the following input: `rel_method = 'auto'`

* The `rel_xtrm_type` argument takes a string, either `'low'` or `'both'` or `'high'`. It indicates which region of the response variable `y` should be considered rare or a "minority", when `rel_method = 'auto'`. In this example, we input `'high'` (default is `'both'`) in the interest of over-sampling high "minority" values in `y` with the following input: `rel_xtrm_type = 'high'`

* The `rel_coef` argument takes a positive real number. It specifies the box plot coefficient used to automatically determine extreme and therefore rare "minority" values in `y`, when `rel_method = 'auto'`. In this example, we input `2.25` (default is `1.50`) to increase the box plot extremes with the following input: `rel_coef = 2.25`

In [22]:
rel_method = 'auto'      # string ('auto' or 'manual')
rel_xtrm_type = 'high'   # string ('low' or 'both' or 'high')
rel_coef = 2.25           # positive real number (0 < R)

## Evaluation

In this section, we present the evaluation metrics available in the package. These metrics can be used to evaluate the performance of models while the dataset is imbalanced. In this example, we are only going to use regression metrics. 
This package includes the following evaluation metrics for regression:

* `phi_weighted_r2` : Calculates the R^2 score between 'y' and 'y_pred' with weighting by `phi`.

* `phi_weighted_mse` : Calculates the mean squared error between 'y' and 'y_pred' with weighting by `phi`.

* `phi_weighted_mae` : Calculates the mean absolute error between 'y' and 'y_pred' with weighting by `phi`.

* `phi_weighted_root_mse` : Calculates the root mean squared error between 'y' and 'y_pred' with weighting by `phi`.

* `ser_t` : Calculates the Squared error-relevance values between 'y' and 'y_pred' with weighting by `phi` at thershold 't'.

* `sera` : Calculates the Squared error-relevance areas (SERA) between 'y' and 'y_pred'.


But first, we are going to import `mean_squared_error`, `mean_absolute_error` and `r2` from sklearn package to see their evaluation. 

In [23]:
# Calculate mean_squared_error
mse = mean_squared_error(y_test , y_pred )
print("Mean Squared Error:", mse)

# Calculate mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae) 

# Calculate r2_score
r2 = r2_score(y_test, y_pred)
print("R2:", r2)            

Mean Squared Error: 8563763.020316727
Mean Absolute Error: 1889.483563706342
R2: 0.861074610723878


In next part, we have used the metrics functions from our package, "ImbalancedDomainsMetrics".

In [24]:
# Calculate phi_weighted_mse. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type and rel_coef.
wmse = rm.phi_weighted_mse (y_test , y_pred, method = rel_method,  xtrm_type = rel_xtrm_type, coef = rel_coef)
print("Weighted Mean Squared Error:", wmse)

# Calculate phi_weighted_mae. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type and rel_coef.
wmae = rm.phi_weighted_mae (y_test , y_pred, method = rel_method,  xtrm_type = rel_xtrm_type, coef = rel_coef)
print("Weighted Mean Absolute Error:", wmae)

# Calculate phi_weighted_r2. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type and rel_coef.
wr2 = rm.phi_weighted_r2 (y_test , y_pred, method = rel_method,  xtrm_type = rel_xtrm_type, coef = rel_coef)
print("Weighted R2:", wr2)

Weighted Mean Squared Error: 36496668.2944312
Weighted Mean Absolute Error: 4667.704153151121
Weighted R2: 0.5690850437151642


In [25]:
# Calculate phi_weighted_root_mse. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type and rel_coef.
wrmse = rm.phi_weighted_root_mse (y_test , y_pred, method = rel_method,  xtrm_type = rel_xtrm_type, coef = rel_coef)
print("Weighted Root Mean Squared Error:", wrmse)

Weighted Root Mean Squared Error: 6041.2472465899955


In [26]:
# Calculate ser_t. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type and rel_coef. The threshold is defined by the user. In this case threshold is .055.
threshold = .055
ser_t= rm.ser_t(y_test, y_pred,threshold, method = rel_method,  xtrm_type = rel_xtrm_type, coef = rel_coef)
print("Ser("+str(threshold)+"):", ser_t)

# Calculate ser_t. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type and rel_coef. The threshold is defined by the user. In this case threshold is .3.
threshold = .7
ser_t= rm.ser_t(y_test, y_pred,threshold, method = rel_method,  xtrm_type = rel_xtrm_type, coef = rel_coef)
print("Ser("+str(threshold)+"):", ser_t)

# Calculate sera. Here the weight calculated by phi relevance function is done by passig rel_method,rel_xtrm_type and rel_coef.
sera= rm.sera(y_test, y_pred, method = rel_method,  xtrm_type = rel_xtrm_type, coef = rel_coef)
print("Sera:", sera)


Ser(0.055): 389224729.1435278
Ser(0.7): 339960088.8710556
Sera: 353971321.3459863


## Optional
Users can also calculate their own weights by using `phi` relevance function. Here rel_method,rel_xtrm_type and rel_coef are passed as arguments.

In [27]:
# Calculate weight using phi relevance function.
rm.calculate_phi(y_test, method = rel_method,  xtrm_type = rel_xtrm_type, coef = rel_coef)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.13848244100583726,
 0.18337780358675157,
 0.04196413114218029,
 0.18087482561036652,
 0.03945601455740089,
 0.0,
 0.0,
 0.0,
 0.0,
 0.07494995622334627,
 0.0,
 0.521894399354412,
 0.042091362119307585,
 0.20370497144986532,
 4.0341219793986274e-05,
 0.057525567013688446,
 0.0,
 0.8010252224620417,
 0.0,
 0.0,
 0.5609042748272306,
 1.0,
 0.9971122824102371,
 0.021713262897932663,
 0.0,
 0.00272499020271495,
 0.0,
 1.0,
 0.3362843053520096,
 0.30180565969677237,
 0.30037763444399274,
 0.0,
 0.0,
 0.018650138168172514,
 0.0,
 0.0,
 0.0952508579917465,
 0.0,
 0.2642992872345165,
 0.0,
 0.0,
 0.0,
 0.0005271767069539197,
 0.0,
 0.0,
 0.0,
 0.20521924265020677,
 0.014731408276826882,
 0.0,
 0.0,
 0.005991127738466844,
 0.0,
 0.053742437448473596,
 1.0,
 0.002340421744713518,
 1.0,
 0.009898585426489859,
 0.2003868823667253]

## Conclusion

In this package, we have presented a set of evaluation metrics specifically designed for imbalanced domains. Our package, "ImbalancedDomainsMetrics", provides a comprehensive set of evaluation metrics to assess the performance of machine learning models trained on imbalanced datasets.

Our package includes several evaluation metrics that address the challenges of imbalanced domains. These metrics can provide a more accurate assessment of model performance than traditional metrics, which can be misleading in imbalanced domains.

To learn more about our package, please refer to the documentation, which includes detailed descriptions of all the available metrics and their usage.

